[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adasegroup/neural_prophet/blob/master/example_notebooks/trend_peyton_manning.ipynb)

# Fitting a changing trend

We will use the time series of the log daily page views for the Wikipedia page for Peyton Manning as an example to illustrate how to fit a changing trend. 

First, we load the data:

In [3]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/adasegroup/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes
    data_location = "https://raw.githubusercontent.com/adasegroup/neural_prophet/master/"
else:
    data_location = "../"

In [4]:
import pandas as pd
from neuralprophet import NeuralProphet

In [5]:
df = pd.read_csv(data_location + "example_data/wp_log_peyton_manning.csv")
df.head(3)

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677


Now we can fit an initial model without any customizations.

We specify the data frequency to be daily. The model will remember this later when we predict into the future.

In [ ]:
m = NeuralProphet()
metrics = m.fit(df, freq="D")
metrics.head(3)

INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 91


  0%|          | 0/273 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.97E-01, min: 5.82E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 4.05E-01
  0%|                                                                                           | 0/91 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - D:\ProgramData\anaconda3\envs\np3\lib\site-packages\pytorch_lightning\utilities\distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name          | Type          | Params
------------------------------------------------
0 | season_params | ParameterDict | 18    
1 | loss_func     | SmoothL1Loss  | 0     
------------------------------------------------
31        Trainable params
0         Non-trainable params
31        Total params
0.000     Total estimated model params size (MB)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

The returned metrics dataframe contains recoded metrics for each training epoch.

Next, we create a dataframe to predict on. 
Here, we specify that we want to predict one year into the future and that we want to include the entire history.

In [ ]:
future = m.make_future_dataframe(df, periods=365, n_historic_predictions=len(df))
future.tail(3)

Note: 'y' and 'y_scaled' are not given for the period extending into the future, as we do not know their true values.

In [ ]:
forecast = m.predict(future)
print(list(forecast.columns))

The returned forecast dataframe contains the original datestamps, 'y' values, the predicted 'yhat' values, residuals and all the individual model components.

In [ ]:
# plots the model predictions
fig1 = m.plot(forecast)

In [ ]:
# plots the individual forecast components for the given time period.
# fig = m.plot_components(forecast, residuals=True)

In [ ]:
# visualizes the model parameters.
fig2 = m.plot_parameters()

# Adjusting Trend

The default values work fairly well in this example. However, the default of 5 changepoints may not be adequate if the actual change in trend happens to fall in a region between the points. 

## Increasing Trend Flexibility
We can address this by increasing the number of changepoints, giving the trend more flexibility, at the danger of overfitting.

Let's try what happens if we increase the number of changepoints to 30.
Additionally, we can increase the range of data on which we fit trend changepoints to only exlude the last 10 percent (default is 20 percent).

In [ ]:
m = NeuralProphet(
    n_changepoints=30,
    changepoints_range=0.90,    
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_parameters()

Looking at the trend rate changes it becomes evident that the trend is overfitting to short-term fluctuations. 

## Automatic trendpoint selection
By adding regularization, we can achieve an automatic selection of the most relevant changepoints and draw the rate changes of other points close to zero. 


In [ ]:
m = NeuralProphet(
    n_changepoints=30,
    trend_reg=1.00,
    changepoints_range=0.90,    
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_parameters()

Now the model selects only a few relevant trend changepoints, drawing the rest closer to zero.

## Manual Trend Changepoints
You can also manually specify the trend changepoints.

Note: A changepoint will always be added at the beginning. You can ignore it.

In [ ]:
m = NeuralProphet(
    changepoints=['2012-01-01', '2014-01-01'],
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_parameters()

## Fine-tuning Trend Flexibility
We can adjust the regularization strength to get more or less points with a non-zero rate change.

Note: for too high regularization strengths, the model fitting process becomes unstable.

In [ ]:
m = NeuralProphet(
    n_changepoints=30,
    trend_reg=3.00,
    changepoints_range=0.90,   
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_parameters()